In [1]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [12]:
traced_script_module.save("traced_script_module.pt")

In [2]:
print(type(traced_script_module))

<class 'torch.jit._trace.TopLevelTracedModule'>


In [2]:
output = traced_script_module(torch.ones(1, 3, 224, 224))

In [4]:
print(output[0,:5])

tensor([-0.4811,  0.6819, -0.0026, -0.1308,  0.3430], grad_fn=<SliceBackward>)


In [13]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

In [14]:
sm.save("my_module.pt")

In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.8556,  0.9050,  0.1528,  0.2607],
         [ 0.8860,  0.9079, -0.1406,  0.8220],
         [ 0.7556,  0.8808, -0.4993, -0.0300]], grad_fn=<TanhBackward>),
 tensor([[ 0.8556,  0.9050,  0.1528,  0.2607],
         [ 0.8860,  0.9079, -0.1406,  0.8220],
         [ 0.7556,  0.8808, -0.4993, -0.0300]], grad_fn=<TanhBackward>))

In [8]:
my_cell(x,h)
traced_cell(x, h)

(tensor([[ 0.8556,  0.9050,  0.1528,  0.2607],
         [ 0.8860,  0.9079, -0.1406,  0.8220],
         [ 0.7556,  0.8808, -0.4993, -0.0300]], grad_fn=<TanhBackward>),
 tensor([[ 0.8556,  0.9050,  0.1528,  0.2607],
         [ 0.8860,  0.9079, -0.1406,  0.8220],
         [ 0.7556,  0.8808, -0.4993, -0.0300]], grad_fn=<TanhBackward>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %input : Float(3:4, 4:1, requires_grad=0, device=cpu),
      %h : Float(3:4, 4:1, requires_grad=0, device=cpu)):
  %19 : __torch__.torch.nn.modules.linear.___torch_mangle_127.Linear = prim::GetAttr[name="linear"](%self.1)
  %21 : Tensor = prim::CallMethod[name="forward"](%19, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-5-1f6e08af67d0>:7:0
  %13 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::add(%21, %h, %12) # <ipython-input-5-1f6e08af67d0>:7:0
  %14 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::tanh(%13) # <ipython-input-5-1f6e08af67d0>:7:0
  %15 : (Float(3:4, 4:1, requires_grad=1, device=cpu), Float(3:4, 4:1, requires_grad=1, device=cpu)) = prim::TupleConstruct(%14, %14)
  return (%15)



In [7]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)
  _1 = torch.tanh(_0)
  return (_1, _1)



In [11]:
from babel.numbers import parse_decimal
x = '6.3'
val = float(parse_decimal('8', locale='en_US'))
print(val)

8.0


In [12]:
import torch
import torchvision

dummy_input = torch.randn(10, 3, 224, 224, device='cpu')
model = torchvision.models.alexnet(pretrained=True).cpu()

# Providing input and output names sets the display names for values
# within the model's graph. Setting these does not change the semantics
# of the graph; it is only for readability.
#
# The inputs to the network consist of the flat list of inputs (i.e.
# the values you would pass to the forward() method) followed by the
# flat list of parameters. You can partially specify names, i.e. provide
# a list here shorter than the number of inputs to the model, and we will
# only set that subset of names, starting from the beginning.
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%actual_input_1 : Float(10:150528, 3:50176, 224:224, 224:1, requires_grad=0, device=cpu),
      %learned_0 : Float(64:363, 3:121, 11:11, 11:1, requires_grad=1, device=cpu),
      %learned_1 : Float(64:1, requires_grad=1, device=cpu),
      %learned_2 : Float(192:1600, 64:25, 5:5, 5:1, requires_grad=1, device=cpu),
      %learned_3 : Float(192:1, requires_grad=1, device=cpu),
      %learned_4 : Float(384:1728, 192:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %learned_5 : Float(384:1, requires_grad=1, device=cpu),
      %learned_6 : Float(256:3456, 384:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %learned_7 : Float(256:1, requires_grad=1, device=cpu),
      %learned_8 : Float(256:2304, 256:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %learned_9 : Float(256:1, requires_grad=1, device=cpu),
      %learned_10 : Float(4096:9216, 9216:1, requires_grad=1, device=cpu),
      %learned_11 : Float(4096:1, requires_grad=1, device=cpu),
      %learned_12 : Float(4096:4096, 4096:1, 

In [16]:
import caffe2.python.onnx.backend as backend
import numpy as np

rep = backend.prepare(model, device="CPU") # or "CPU"
# For the Caffe2 backend:
#     rep.predict_net is the Caffe2 protobuf for the network
#     rep.workspace is the Caffe2 workspace for the network
#       (see the class caffe2.python.onnx.backend.Workspace)
outputs = rep.run(np.random.randn(10, 3, 224, 224).astype(np.float32))
# To run networks with more than one input, pass a tuple
# rather than a single numpy ndarray.
print(outputs[0])

ModuleAttributeError: 'AlexNet' object has no attribute 'SerializeToString'

In [17]:
import onnxruntime as ort

ort_session = ort.InferenceSession('alexnet.onnx')

outputs = ort_session.run(None, {'actual_input_1': np.random.randn(10, 3, 224, 224).astype(np.float32)})

print(outputs[0])

[[-0.15288618 -1.7744154  -1.772217   ... -1.2685202  -1.1067306
   1.457113  ]
 [-0.06498257 -0.9908374  -1.1324918  ... -0.95935225 -0.97419715
   1.2393706 ]
 [-0.05368517 -1.7085843  -1.1514349  ... -1.3531338  -0.936008
   1.0077317 ]
 ...
 [-0.09625517 -1.5160254  -1.2915084  ... -1.3142797  -0.89416385
   0.889783  ]
 [ 0.09142481 -1.2674376  -1.1974701  ... -0.8329861  -0.82525307
   1.1199676 ]
 [ 0.04747842 -1.453126   -1.6183759  ... -1.3628887  -0.90458643
   0.7608837 ]]


In [15]:
print(type(model))

<class 'torchvision.models.alexnet.AlexNet'>


In [38]:
import torch

# Trace-based only

class LoopModel(torch.nn.Module):
    def forward(self, x, y):
        for i in range(y):
            x = x + i
        return x

loopmodel = LoopModel()
dummy_input = torch.ones(2, 3, dtype=torch.long)
loop_count = torch.tensor(5, dtype=torch.long)

torch.onnx.export(loopmodel, (dummy_input, loop_count), 'loop.onnx', verbose=True,input_names=['input_data', 'loop_range'])


graph(%input_data : Long(2:3, 3:1, requires_grad=0, device=cpu)):
  %2 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={0}]()
  %3 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Add(%input_data, %2)
  %4 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %5 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Add(%3, %4)
  %6 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={2}]()
  %7 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Add(%5, %6)
  %8 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={3}]()
  %9 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Add(%7, %8)
  %10 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={4}]()
  %11 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Add(%9, %10)
  return (%11)



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  import sys


In [42]:
import onnxruntime as ort
ort_sess = ort.InferenceSession('loop.onnx')
outputs = ort_sess.run(None, {'input_data': dummy_input.numpy()})
print(outputs)

[array([[11, 11, 11],
       [11, 11, 11]], dtype=int64)]


In [22]:
@torch.jit.script
def loop(x, y):
    for i in range(int(y)):
        x = x + i
    return x

class LoopModel2(torch.nn.Module):
    def forward(self, x, y):
        return loop(x, y)

lpmodel = LoopModel2()

dummy_input = torch.ones(2, 3, dtype=torch.long)
loop_count = torch.tensor(5, dtype=torch.long)
torch.onnx.export(lpmodel, (dummy_input, loop_count), 'lploop.onnx', verbose=True,
                  input_names=['input_data', 'loop_range'])

graph(%input_data : Long(2:3, 3:1, requires_grad=0, device=cpu),
      %loop_range : Long(requires_grad=0, device=cpu),
      %10 : Bool(requires_grad=0, device=cpu)):
  %2 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %4 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Loop(%loop_range, %10, %input_data) # <ipython-input-22-a4f9461791d8>:3:4
    block0(%i.1 : Long(device=cpu), %cond : bool, %x.6 : Long(2:3, 3:1, requires_grad=0, device=cpu)):
      %8 : LongTensor = onnx::Add(%x.6, %i.1) # <ipython-input-22-a4f9461791d8>:4:12
      %9 : bool = onnx::Cast[to=9](%2)
      -> (%9, %8)
  return (%4)



In [44]:
import onnxruntime as ort
ort_sess = ort.InferenceSession('lploop.onnx')
outputs = ort_sess.run(None, {'input_data': dummy_input.numpy(),'loop_range':np.array(5).astype(np.int64)})
print(outputs)

[array([[11, 11, 11],
       [11, 11, 11]], dtype=int64)]


In [25]:
lpmodel_script= torch.jit.script(lpmodel)

In [47]:
print(lpmodel_script.code)

def forward(self,
    x: Tensor,
    y: Tensor) -> Tensor:
  x0 = x
  for i in range(int(y)):
    x0 = torch.add(x0, i, 1)
  return x0



In [31]:
output = lpmodel_script(dummy_input, loop_count)
torch.onnx.export(lpmodel_script, (dummy_input, loop_count), 'lploop_script.onnx', verbose=True,
                  input_names=['input_data', 'loop_range'],example_outputs=output)

graph(%input_data : Long(2:3, 3:1, requires_grad=0, device=cpu),
      %loop_range : Long(requires_grad=0, device=cpu),
      %10 : Bool(requires_grad=0, device=cpu)):
  %2 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %4 : Long(2:3, 3:1, requires_grad=0, device=cpu) = onnx::Loop(%loop_range, %10, %input_data) # <ipython-input-22-a4f9461791d8>:3:4
    block0(%i.1 : Long(device=cpu), %cond : bool, %x.6 : Long(2:3, 3:1, requires_grad=0, device=cpu)):
      %8 : LongTensor = onnx::Add(%x.6, %i.1) # <ipython-input-22-a4f9461791d8>:4:12
      %9 : bool = onnx::Cast[to=9](%2)
      -> (%9, %8)
  return (%4)



In [46]:
import onnxruntime as ort
ort_sess = ort.InferenceSession('lploop_script.onnx')
outputs = ort_sess.run(None, {'input_data': dummy_input.numpy(),'loop_range':np.array(9).astype(np.int64)})
print(outputs)

[array([[37, 37, 37],
       [37, 37, 37]], dtype=int64)]
